Solving Package delivery using single-agent PPO with a naive feature representation learning: concatenante all the feature in to a single state vector, and multiple robot actions as a multi discrete distribution.

In [ ]:
# %%capture
# !git clone https://github.com/cuongtv312/marl-delivery.git
# %cd marl-delivery
# !pip install -r requirements.txt

In [2]:
%%capture
!pip install stable-baselines3

In [3]:
from env import Environment
import gymnasium as gym
from gymnasium import spaces
import numpy as np

In [1]:
import numpy as np
import torch
from env import Environment
from ppo_agent import PPOAgent
from tqdm import trange

In [10]:
num_episodes = 500
n_robots = 5
n_packages = 20
max_steps = 2000
map_file = "map3.txt"

# Thông số PPO
obs_dim = 454  # 3 (robot) + 5*10 (gói) + 1 (time) + 400 (map 20x20)
act_dim = 15   # 5 hướng * 3 hành động
agent = PPOAgent(obs_dim, act_dim)


In [3]:
# TODO: Modify this one to add more information to the Agents
def convert_state(state, robot_id, max_pkg=10):
    robot = state["robots"][robot_id]
    rx, ry, carrying = robot[0] / 20.0, robot[1] / 20.0, robot[2] / 100.0
    time = np.array([state["time_step"] / 1000.0], dtype=np.float32)

    map_flat = np.array(state["map"], dtype=np.float32).flatten() / 1.0

    packages = state["packages"][:max_pkg]
    pkg_feats = []
    for p in packages:
        sx, sy, dx, dy = p[1]/20.0, p[2]/20.0, p[3]/20.0, p[4]/20.0
        deadline = p[5] / 1000.0
        pkg_feats.extend([sx, sy, dx, dy, deadline])
    while len(pkg_feats) < max_pkg * 5:
        pkg_feats.append(0.0)

    return np.concatenate([[rx, ry, carrying], pkg_feats, time, map_flat])



In [4]:
# TODO: Modify this one to make the agent learn faster

def reward_shaping(r, action):
    shaped = r
    if action[1] == '1':
        shaped += 0.2  # thưởng nhặt
    elif action[1] == '2':
        shaped += 0.5  # thưởng giao
    if action[0] == 'S':
        shaped -= 0.01  # phạt đứng yên
    return shaped



In [5]:
def reward_shaping(r, action):
    shaped = r
    if action[1] == '1':
        shaped += 0.2  # thưởng nhặt
    elif action[1] == '2':
        shaped += 0.5  # thưởng giao
    if action[0] == 'S':
        shaped -= 0.01  # phạt đứng yên
    return shaped


In [6]:
# Avoid to modify the Env class,
# If it is neccessary, you should describe those changes clearly in report and code
class Env(gym.Env):
    def __init__(self, *args, **kwargs):
        super(Env, self).__init__()
        self.env = Environment(*args, **kwargs)

        self.action_space = spaces.multi_discrete.MultiDiscrete([5, 3]*self.env.n_robots)


        self.prev_state = self.env.reset()
        first_state=convert_state(self.prev_state)
        # Define observation space as a dictionary

        self.observation_space = spaces.Box(low=0, high=100, shape=first_state.shape, dtype=np.float32)


        from sklearn.preprocessing import LabelEncoder
        self.le1, self.le2= LabelEncoder(), LabelEncoder()
        self.le1.fit(['S', 'L', 'R', 'U', 'D'])
        self.le2.fit(['0','1', '2'])

    def reset(self, *args, **kwargs):
        self.prev_state = self.env.reset()
        return convert_state(self.prev_state), {}

    def render(self, *args, **kwargs):
        return self.env.render()

    def step(self, action):
        ret = []
        ret.append(self.le1.inverse_transform(action.reshape(-1, 2).T[0]))
        ret.append(self.le2.inverse_transform(action.reshape(-1, 2).T[1]))
        action = list(zip(*ret))

        # You should not modify the infos object
        s, r, done, infos = self.env.step(action)
        new_r = reward_shaping(r, self.env, self.prev_state, action)
        self.prev_state = s
        return convert_state(s), new_r, \
            done, False, infos

NameError: name 'gym' is not defined

In [7]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback


# Parallel environments
vec_env = make_vec_env(lambda: Env('map2.txt', 100, 5, 20, -0.01, 10., 1., 10), n_envs=10)
eval_env = Monitor(Env('map2.txt', 100, 5, 20, -0.01, 10., 1., 10), "ppo_delivery")

eval_callback = EvalCallback(eval_env, best_model_save_path="./best_model/",
                             log_path="./logs/", eval_freq=5000,
                             deterministic=True, render=False)

model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=10000, callback=eval_callback)
model.save("ppo_delivery")


NameError: name 'Env' is not defined

In [8]:
from ppo_agent import PPOAgent
agent = PPOAgent(obs_dim=203, act_dim=15)  # sửa số chiều phù hợp

for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        actions = []
        log_probs = []
        entropies = []
        obs_list = []

        for i in range(len(state["robots"])):
            obs = convert_state(state, i)
            action, log_prob, entropy = agent.select_action(obs)
            obs_list.append(obs)
            actions.append((action // 3, str(action % 3)))  # giải mã action
            log_probs.append(log_prob)
            entropies.append(entropy)

        next_state, rewards, done, info = env.step(actions)

        for i in range(len(state["robots"])):
            next_obs = convert_state(next_state, i)
            r = rewards[i]
            agent.store(obs_list[i], actions[i][0] * 3 + int(actions[i][1]), r, next_obs, done, log_probs[i])
            total_reward += r

        state = next_state

    agent.train()
    print(f"Episode {episode} | Reward: {total_reward}")


NameError: name 'episodes' is not defined

In [9]:
obs,_ = eval_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, _, info = eval_env.step(action)
    #print('='*10)
    #eval_env.unwrapped.env.render()
    if dones:
        break

print(info)

{'total_reward': -3.299999999999999, 'total_time_steps': 100, 'episode': {'r': -3.3, 'l': 100, 't': 168.352014}}


In [10]:
!pip freeze | grep stable_baselines3

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
for ep in trange(num_episodes):
    env = Environment(map_file=map_file, max_time_steps=max_steps,
                      n_robots=n_robots, n_packages=n_packages)
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        actions = []
        log_probs = []
        obs_list = []

        for i in range(n_robots):
            obs = convert_state(state, i)
            act_flat, log_prob, _ = agent.select_action(obs)
            move = ['S', 'L', 'R', 'U', 'D'][act_flat // 3]
            act = str(act_flat % 3)
            actions.append((move, act))
            obs_list.append(obs)
            log_probs.append(log_prob)

        next_state, reward, done, infos = env.step(actions)

        for i in range(n_robots):
            shaped_reward = reward_shaping(reward, actions[i])  
            next_obs = convert_state(next_state, i)
            flat_action = ['S', 'L', 'R', 'U', 'D'].index(actions[i][0]) * 3 + int(actions[i][1])
            agent.store(obs_list[i], flat_action, shaped_reward, next_obs, done, log_probs[i])
            total_reward += shaped_reward

        state = next_state

    agent.train()
    print(f"Episode {ep} | Total reward: {total_reward:.2f}")


  0%|          | 0/500 [00:00<?, ?it/s]c:\Users\trangdo\Documents\HocTap\TrenLop\RL\groupProject\marl-delivery\ppo_agent.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  obs = torch.FloatTensor(obs)
  0%|          | 1/500 [00:08<1:07:44,  8.15s/it]

Episode 0 | Total reward: 2117.28


  0%|          | 2/500 [00:15<1:05:09,  7.85s/it]

Episode 1 | Total reward: 2120.58


  1%|          | 3/500 [00:24<1:08:01,  8.21s/it]

...

Episode 124 | Total reward: 2060.20


 25%|██▌       | 126/500 [16:46<53:51,  8.64s/it]

Episode 125 | Total reward: 2061.70


 25%|██▌       | 127/500 [16:55<52:59,  8.52s/it]

Episode 126 | Total reward: 2126.33
